In [ ]:
import requests
import time
import pandas as pd

# API endpoint
url = "https://api.usaspending.gov/api/v2/recipient/"

# Headers for the request
headers = {
    "Accept": "application/json, text/plain, /",
    "Content-Type": "application/json",
    "User-Agent": "Mozilla/5.0"
}

# Initialize variables for pagination
company_names = []
uei_names = []
gr_amt = []
page = 1
limit = 1000  # Adjust limit if needed (API allows up to 100 per page)

# Loop through pages
while True:
    # Construct the payload
    payload = {
        "order": "desc",
        "sort": "amount",
        "page": page,
        "limit": limit,
        "award_type": "grants"
    }
    
    # Send the POST request
    response = requests.post(url, headers=headers, json=payload)
    # print(response.text)
    # time.sleep(2)
    
    if response.status_code != 200:
        print(f"Error: {response.status_code}, {response.text}")
        break
    
    # Parse the response
    data = response.json()
    results = data.get("results", [])
    
    # Check if results are empty
    if not results:
        print("No more results found.")
        break
    
    # Extract company names and append to the list
    for record in results:
        company_name = record.get("name")
        
        uei = record.get("uei")
        gr = record.get("amount")

        if company_name:  # Ensure the name is not None
            company_names.append(company_name)
            uei_names.append(uei)
            gr_amt.append(gr)
        # print(company_name)
        # time.sleep(5)
    
    # Pagination metadata
    total_results = data.get("page_metadata", {}).get("total", 0)
    has_next = data.get("page_metadata", {}).get("hasNext", False)
    
    print(f"Page {page} fetched with {len(results)} records. Total company names collected so far: {len(company_names)} of {total_results}.")
    
    # Check if there is a next page
    if not has_next:
        break
    
    # Increment the page number
    page += 1

parse_dict = {'company_names':company_names, 'uei':uei_names , 'grants': gr_amt}
pd.DataFrame(parse_dict).to_csv("../data/grant_details.csv", index=False)

# Print the collected company names
print(f"Total company names collected: {len(company_names)}")
print(company_names)